In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import librosa
import soundfile as sf
from tqdm import tqdm
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import (Dense, Flatten, Reshape, Concatenate, Conv2D,
                                     UpSampling2D, BatchNormalization, MaxPooling2D, Conv2DTranspose)
tfd = tfp.distributions
tfb = tfp.bijectors
tfpl = tfp.layers

In [59]:
def sample_point_from_normal_distribution(args):
    mu, log_variance = args
    epsilon = K.random_normal(shape=K.shape(mu), mean=0., stddev=1.)
    return mu + K.exp(log_variance / 2) * epsilon

In [60]:
def vae_loss(y_target, y_predicted, mu, log_variance):
    reconstruction_loss = K.mean(K.square(y_target - y_predicted), axis=[1, 2, 3])
    kl_loss = -0.5 * K.sum(1 + log_variance - K.square(mu) - K.exp(log_variance), axis=1)
    combined_loss = reconstruction_loss * 1.0 + kl_loss
    return combined_loss

In [61]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, Flatten, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

def get_encoder(latent_dim):
    encoder_input = Input(shape=(256, 64, 1), name='encoder_input')
    
    # First convolutional block
    x = Conv2D(filters=512, kernel_size=3, strides=2, padding='same')(encoder_input)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    
    # Second convolutional block
    x = Conv2D(filters=256, kernel_size=3, strides=2, padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    
    # Third convolutional block
    x = Conv2D(filters=128, kernel_size=3, strides=2, padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    
    # Fourth convolutional block
    x = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    
    # Fifth convolutional block
    x = Conv2D(filters=32, kernel_size=3, strides=(2,1), padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    
    # Flatten data and add bottleneck with Gaussian sampling
    x = Flatten()(x)
    mu = Dense(latent_dim, name="mu")(x)
    log_variance = Dense(latent_dim, name="log_variance")(x)
    
    # Lambda layer for sampling using reparameterization trick
    encoder_output = Lambda(sample_point_from_normal_distribution, name="encoder_output")([mu, log_variance])
    
    # Create model
    encoder = Model(encoder_input, [encoder_output, mu, log_variance], name="encoder")
    
    return encoder


In [62]:
def get_encoder(latent_dim):
    # Define the input
    encoder_input = Input(shape=(256, 64, 1), name='encoder_input')

    # Convolutional layers
    x = Conv2D(filters=512, kernel_size=3, strides=2, padding='same')(encoder_input)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    
    # Continue with the rest of the blocks
    ...
    
    # Flatten data and create bottleneck with Gaussian sampling
    x = Flatten()(x)
    mu = Dense(latent_dim, name="mu")(x)
    log_variance = Dense(latent_dim, name="log_variance")(x)

    # Lambda layer for reparameterization trick
    encoder_output = Lambda(sample_point_from_normal_distribution, name="encoder_output")([mu, log_variance])
    
    # Return encoder with the output tensors
    encoder = Model(encoder_input, [encoder_output, mu, log_variance], name="encoder")
    
    return encoder


In [63]:
def calculate_combined_loss(y_target, y_predicted, mu, log_variance, reconstruction_loss_weight):
    reconstruction_loss = K.mean(K.square(y_target - y_predicted), axis=[1, 2, 3])
    kl_loss = -0.5 * K.sum(1 + log_variance - K.square(mu) - K.exp(log_variance), axis=1)
    combined_loss = reconstruction_loss_weight * reconstruction_loss + kl_loss
    return combined_loss

In [75]:
latent_dim = 128
learning_rate = 0.0005 
reconstruction_loss_weight = 1000000

encoder = get_encoder(latent_dim)
decoder = get_decoder(latent_dim)

        # VAE architecture
vae_input = Input(shape=(256, 64, 1), name='vae_input')
latent_representation = encoder(vae_input)
decoded_output = decoder(latent_representation)

        # Custom loss calculation in a lambda layer to extract mu and log_variance
mu = encoder.get_layer('mu').output
log_variance = encoder.get_layer('log_variance').output

        # Create VAE model
vae = Model(vae_input, decoded_output, name='vae')

        # Compile with custom loss
vae.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=lambda y_true, y_pred: calculate_combined_loss(y_true, y_pred, mu, log_variance, reconstruction_loss_weight),
    metrics=['mse']
)

vae.summary()

ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[2097152,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:AddV2] name: 

In [ ]:
import os
import numpy as np

directory_path = "spectrograms"

loaded_dict = {}

file_list = os.listdir(directory_path)

for file_name in file_list:
    if file_name.endswith('.npy'):  
        file_path = os.path.join(directory_path, file_name)  
        array = np.load(file_path)  
        array_with_new_axis = np.expand_dims(array, axis=-1)
        loaded_dict[file_name] = array_with_new_axis


In [76]:
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np

checkpoint_cb = ModelCheckpoint(
    'best_model_MNIST.keras', 
    save_best_only=True,
    monitor='val_loss', 
    mode='min' 
)

def data_generator(data_dict, batch_size):
    keys = list(data_dict.keys())  
    while True:
        np.random.shuffle(keys)  
        for i in range(0, len(keys), batch_size):
            key_batch = keys[i:i + batch_size]
            x_batch = np.array([data_dict[key] for key in key_batch])
            yield x_batch, x_batch 
            
batch_size = 16
train_gen = data_generator(loaded_dict, batch_size)

train_steps = len(loaded_dict) // batch_size
history = vae.fit(
    x=train_gen,  
    steps_per_epoch=train_steps,  
    epochs=15,  
    callbacks=[checkpoint_cb]  
)

Epoch 1/15


ValueError: in user code:

    File "C:\Users\mayal\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\mayal\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\mayal\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\mayal\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\mayal\AppData\Roaming\Python\Python310\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\mayal\AppData\Roaming\Python\Python310\

In [ ]:
import librosa
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import tensorflow as tf 

def inverse_normalize_spectrogram(normalized_spectrogram, min_val, max_val):
    return normalized_spectrogram * (max_val - min_val) + min_val

def reconstruct_audio(stft, output_path, min_val, max_val):
    stft = stft[:, :, 0]
    magnitude_db = inverse_normalize_spectrogram(stft, min_val, max_val)
    magnitude_linear = librosa.db_to_amplitude(magnitude_db)
    y_reconstructed = librosa.istft(magnitude_linear, hop_length=256)
    sf.write(output_path, y_reconstructed, 22050)
    
    plt.figure(figsize=(10, 4))
    D = librosa.amplitude_to_db(np.abs(librosa.stft(y_reconstructed, n_fft=512, hop_length=256)), ref=np.max)
    librosa.display.specshow(D, sr=22050, hop_length=256, x_axis='time', y_axis='linear')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Reconstructed Spectrogram')
    plt.tight_layout()
    plt.show()

In [ ]:
def generate_music(prior, decoder, n_samples):
    z = prior.sample(n_samples)
    return decoder(z).mean()

n_samples = 5
sm = generate_music(prior, decoder, n_samples)
reconstruct_audio(sm[1], "originalu.wav", -80, 0)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf

def reconstruct(encoder, decoder, batch_of_images):
    latent_representations = encoder(batch_of_images)  
    distribution = decoder(latent_representations)  
    reconstructed_image_tensor = distribution.mean()  
    return reconstructed_image_tensor

n_reconstructions = 5
keys = list(loaded_dict.keys())
random_keys = np.random.choice(keys, n_reconstructions, replace=False)
batch_of_images = np.array([loaded_dict[key] for key in random_keys])
reconstructions = reconstruct(encoder, decoder, batch_of_images)
reconstructed_images_np = reconstructions.numpy()

def compare_reconstructed(reconstructed_images_np, min_max_dict, random_keys):
    for i, image in enumerate(reconstructed_images_np):
        file_name = random_keys[i]
        base_file_name = os.path.splitext(file_name)[0]

        min_val = min_max_dict[base_file_name]['min']
        max_val = min_max_dict[base_file_name]['max']

        reconstruct_audio(image, f"reconstructed_{file_name}.wav", min_val, max_val)
        reconstruct_audio(loaded_dict[file_name], f"original_{file_name}.wav", min_val, max_val)

min_max_dict = np.load('min_max_values.npy', allow_pickle=True).item()
compare_reconstructed(reconstructed_images_np, min_max_dict, random_keys)